In [64]:
"""
author: abrito, brito.anthony@epa.gov
purpose: rename and add columns to ATSDR MRL 2022 table (table extracted manually from pdf)(TOXVAL-322)
"""
import pandas as pd
import numpy as np

In [65]:
#load in the manually made table
MRL_table = pd.read_excel(r"O:\Public\rsayre01\abrito\ATSDR MRLs 2022\Book1 - Copy.xlsx")

In [66]:
#rename 'Duration' column to 'Duration_category'
MRL_table.rename(columns = {'Duration':'Duration_category'}, inplace = True)

In [67]:
#removes leading and trailing spaces in dataframe
MRL_table = MRL_table.replace(r"^ +| +$", r"", regex=True)

In [68]:
#rename abbreviations for 'duration_category' to its full word
MRL_table['Duration_category'].replace({'Acute': 'Acute', 'Int.': 'Intermediate',  'Chr.' : 'Chronic'}, inplace = True)

In [69]:
#Include "duration_value" and "duration_units" columns mapped from the renamed "duration_category" column
#using the last page dictionary of what "Acute", "Intermediate", and "Chronic" represent
#For Duration, Acute = 1 to 14 days, Intermediate = 15 to 364 days, and Chronic = 1 year or longer
conditions = [
    (MRL_table['Duration_category'] == 'Acute'),
    (MRL_table['Duration_category'] == 'Intermediate'),
    (MRL_table['Duration_category'] == 'Chronic'),
]

values = ['1 to 14', '15 to 364', '1 or longer']
units = ['days', 'days', 'years']
MRL_table['Duration_value'] = np.select(conditions, values)
MRL_table['Duration_unit'] = np.select(conditions, units)


In [70]:
#move 'Duration_value' and 'Duration_units' columns from end of table to right after 'Duration_category'
move_dur_value = MRL_table.pop('Duration_value')
move_dur_units = MRL_table.pop('Duration_unit')
MRL_table.insert(3, "Duration_value", move_dur_value)
MRL_table.insert(4, "Duration_units", move_dur_units)


In [71]:
#replace 'Route' abbreviations with its full word
MRL_table['Route'].replace({'Oral': 'Oral', 'Rad.': 'External Radiation',  'Inh.' : 'Inhalation'}, inplace = True)

In [72]:
#Split 'MRL' column into 'MRL_value' and 'MRL_unit', drip the old 'MRL' column
MRL_table[['MRL_value', 'MRL_unit']] = MRL_table.MRL.str.split(" ", expand = True)
MRL_table.drop('MRL', axis=1, inplace=True)

#move the new MRL columns from the end to the same position the old MRL column was
move_mrl_value = MRL_table.pop('MRL_value')
move_mrl_units = MRL_table.pop('MRL_unit')
MRL_table.insert(5, "MRL_value", move_mrl_value)
MRL_table.insert(6, "MRL_unit", move_mrl_units)


In [73]:
#replace 'Endpoint' abbreviations with its full word, except for 'Lymphor.' value, I do not know its unabbreviated version
#MRL_table['Endpoint'].replace({'Hepatic' : 'Hepatic', 'Neurol.' : 'Neurological', 'Resp.' : 'Respiratory', 'Develop.' : 'Developmental', 'Reprod.' : 'Reproductive', 'Hemato.' : 'Hematological', 'Renal' : 'Renal', 'Immuno.' : 'Immunological', 'Other': 'Other', 'Body Wt.' : 'Body Weight', 'Gastro.' : 'Gastrological', 'Metab.' : 'Metabolic', 'Dermal' : 'Dermal', 'Musculo.' : 'Musculoskeletal', 'Endocr.' : 'Endocrinological', 'Ocular' : 'Ocular', 'Neurol.,Develop.' : 'Neurological, Developmental', 'Neurol.,Hepatic' : 'Neurological, Hepatic', 'Hepatic,Endocr.' : 'Hepatic, Endocrinological', 'Lymphor.' : 'Lymphor.', 'Hepatic, Renal' : 'Hepatic, Renal'})

In [74]:
#Add url column with 'https://wwwn.cdc.gov/TSP/MRLS/mrlsListing.aspx' as the value
MRL_table['url'] = 'https://wwwn.cdc.gov/TSP/MRLS/mrlsListing.aspx'

In [76]:
#! trichloroethylene 438:446

                  Name       Route Duration_category Duration_value  \
438  TRICHLOROETHYLENE  Inhalation           Chronic    1 or longer   
439  TRICHLOROETHYLENE  Inhalation           Chronic    1 or longer   
440  TRICHLOROETHYLENE        Oral      Intermediate      15 to 364   
441  TRICHLOROETHYLENE        Oral      Intermediate      15 to 364   
442  TRICHLOROETHYLENE        Oral      Intermediate      15 to 364   
443  TRICHLOROETHYLENE        Oral           Chronic    1 or longer   
444  TRICHLOROETHYLENE        Oral           Chronic    1 or longer   
445  TRICHLOROETHYLENE        Oral           Chronic    1 or longer   

    Duration_units MRL_value   MRL_unit  Total Factors*  Endpoint Status  \
438          years    0.0004        ppm              10  Develop.  Final   
439          years    0.0004        ppm             100   Immuno.  Final   
440           days    0.0005  mg/kg/day              10  Develop.  Final   
441           days    0.0005  mg/kg/day             100 

In [75]:
MRL_table.to_excel(r'O:\Public\rsayre01\abrito\ATSDR MRLs 2022\atsdr_mrls_2022_raw.xlsx', index = False)